This notebook shows different visualizations of the algorithms performances

In [ ]:
import config

In [ ]:
import os
import pickle
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Open all the score

In [ ]:
date = '11 March 2019 12:28:49'

In [ ]:
def openFiles(date): 
    path = os.path.join(config.result, date)
    scores = {}
    # Open each dataset
    for dname in os.listdir(path):
        unpick = pickle.load(open(os.path.join(path, dname), 'rb'))
        # Restructure to have a nice pandas dataframe
        openScore = {(category, method, metric): unpick["Score"][method][category][metric] 
                        for method in unpick["Score"] 
                        for category in unpick["Score"][method]
                        for metric in unpick["Score"][method][category]}
        openScore.update({("Assignation", method,): unpick["Assignation"][method]
                        for method in unpick["Assignation"]})
        openScore[("Dataset", "N_Classes")] = unpick["N_Classes"]
        openScore[("Dataset", "Constraint")] = unpick["Constraint"]
        openScore[("Dataset", "Labels")] = unpick["Labels"]
        openScore[("Dataset", "Train")] = unpick["Train"]
        openScore[("Dataset", "Name")] = unpick["Name"]
        scores[dname] = openScore
    return pd.DataFrame.from_dict(scores, orient="index")

In [ ]:
scores = openFiles(date)
print("Open {} result files from {}".format(len(scores), date))

In [ ]:
scores.head()

# Comparison

In [ ]:
metric = "Adjusted Rand"
category = "test" # 'all', 'train' or 'test'

## Global performance

Compute the global performances of the different methods

In [ ]:
measure = scores.loc[:, (category, slice(None), metric)]
measure = measure.T.reset_index(level=[0, 2], drop=True).T
sns.boxplot(data = measure, orient="h")
plt.ylabel("Methods")
plt.xlabel(metric)

In [ ]:
sns.violinplot(data = measure, orient="h")
plt.ylabel("Methods")
plt.xlabel(metric)

In [ ]:
measure["Number classes"] = scores[("Dataset", "N_Classes", )]
measure["Number points"] = scores[("Dataset", "Labels", )].apply(lambda x: len(x.values[0]), axis = 1) 
var = [c for c in measure.columns if c not in ["Number classes", "Number points"]]
       
cmap = sns.cubehelix_palette(rot=-.2, as_cmap=True)

grid = sns.PairGrid(data = measure, vars = var)
grid = grid.map_upper(sns.scatterplot, sizes=(10, 200), size=measure["Number points"], hue=measure["Number classes"], palette = cmap, alpha = 0.7)
grid = grid.map_diag(sns.distplot)
grid = grid.map_lower(sns.kdeplot)

## First ranked

Compute and display the rank of the different methods

In [ ]:
# Count rank position
rank = measure[var].rank(axis = 1, method = 'min',ascending = False).apply(lambda x: x.value_counts()).T 
rank = rank.sort_values(by=[1, 2, 3], ascending = False) # Reorder
rank.plot.barh(stacked=True)
plt.xlabel("Number dataset")
plt.ylabel("Methods")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), title="Rank")

# Analysis by method

In [ ]:
method = "lcvqe"
metric = "Adjusted Rand"

## Train vs Test Performances

Compare performances between training and test, it is interesting to notice how our method does not overfit where other methods perform well on training but have bad performances on test.

In [ ]:
measure = scores.loc[:, ([slice(None)], method, metric)]
measure = measure.T.reset_index(level=[1, 2], drop=True).T
measure["Number points"] = scores[("Dataset", "Labels", )].apply(lambda x: len(x.values[0]), axis = 1) 
measure["Number classes"] = scores[("Dataset", "N_Classes", )]
plt.title(method)
plt.grid(alpha = 0.1)
sns.scatterplot(x="train", y="test", sizes=(10, 200), data=measure, 
                size="Number points", hue="Number classes", palette = cmap, alpha = 0.7)
plt.xlim(-0.1,1.1)
plt.ylim(-0.1,1.1)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

## Train vs Test Constraint

Compare the percentage of cosntraint respected on train and test

In [ ]:
import numpy as np
from constraint import verification_constraint

In [ ]:
def percentageConstraint(gtLabels, label, trainIndices, test = False):
    """
        Compute the percentage of constraint respected
        gtLabels -- Ground truth label
        label -- Ground truth label
        trainIndices -- Indices for train
        test -- Compute on test supset if True
    """
    if test:
        trainIndices = [i for i in range(len(gtLabels)) if i not in trainIndices]
    gtConstraint = 2*np.equal.outer(gtLabels[trainIndices], gtLabels[trainIndices]) - 1
    return verification_constraint(gtConstraint, label[trainIndices])    

In [ ]:
measure["% train constraint respected"] = scores.transform(lambda x: percentageConstraint(
                                                                    x[('Dataset', 'Labels', pd.NaT)], 
                                                                    x[('Assignation', method, pd.NaT)], 
                                                                    x[('Dataset', 'Train', pd.NaT)]),
                                                            axis = 1).apply(lambda x: x[0] / np.sum(x))
measure["% test constraint respected"] =  scores.transform(lambda x: percentageConstraint(
                                                                    x[('Dataset', 'Labels', pd.NaT)], 
                                                                    x[('Assignation', method, pd.NaT)], 
                                                                    x[('Dataset', 'Train', pd.NaT)],
                                                                    True),
                                                            axis = 1).apply(lambda x: x[0] / np.sum(x))
plt.title(method)
plt.grid(alpha = 0.1)
sns.scatterplot(x="% train constraint respected", y="% test constraint respected", sizes=(10, 200), data=measure, 
                size="Number points", hue="Number classes", palette = cmap, alpha = 0.7)
plt.xlim(-0.1,1.1)
plt.ylim(-0.1,1.1)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
plt.title(method)
plt.grid(alpha = 0.1)
sns.scatterplot(x="% test constraint respected", y="test", sizes=(10, 200), data=measure, 
                size="Number points", hue="Number classes", palette = cmap, alpha = 0.7)
plt.xlim(-0.1,1.1)
plt.ylim(-0.1,1.1)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

-----

# Evolution

In [ ]:
date = '12 March 2019 16:22:24_evolution'

In [ ]:
scores = openFiles(date)
print("Open {} result files from {}".format(len(scores), date))

In [ ]:
scores[("Dataset", "Percentage", pd.NaT)] = pd.DataFrame.from_dict({p: np.mean(np.abs(scores.loc[p, ("Dataset", "Constraint", pd.NaT)])) for p in scores.index}, orient = "index")

## Performances

In [ ]:
metric = "Adjusted Rand"
category = "test" # 'all', 'train' or 'test'

In [ ]:
measure = scores.loc[:, (category, slice(None), metric)]
measure = measure.T.reset_index(level=[0, 2], drop=True).T
measure.groupby(scores[('Dataset', 'Name', pd.NaT)]).plot()